In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np 
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import os
import cv2
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Load the data
# Load the data
train_data = pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-2/train.csv')

# Prepare the data
X = train_data['file_name']  # Image file names
y = train_data['label']  # Class labels
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# Define data augmentation and preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# Define data augmentation and preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train = pd.DataFrame()
train['file_name'] = X_train 
train['label'] = y_train
# Convert numeric labels to strings
train['label'] = train['label'].astype(str)

# Create the train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    x_col='file_name',
    y_col='label',
    directory='/kaggle/input/extracting-attributes-from-fashion-images-2/train',
    shuffle=True,
    target_size=(100, 100),
    batch_size=32,
    class_mode='sparse'
)
valid = pd.DataFrame()
valid['file_name'] = X_val 
valid['label'] = y_val
# Convert numeric labels to strings
valid['label'] = valid['label'].astype(str)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=valid,
    x_col="file_name",
    y_col="label",
    directory='/kaggle/input/extracting-attributes-from-fashion-images-2/train',
    #subset="validation",  # Use the validation subset of the data
    shuffle=True,
    target_size=(100, 100),  # Update the target size if needed
    batch_size=32,
    class_mode='sparse'
)
# Define the CNN model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)
test = pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-2/sample_submission.csv')
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
test['label'] = test['label'].astype(str)
test_images = test_generator.flow_from_dataframe(
    dataframe=test,
    x_col='file_name',
    y_col='label',
    directory='/kaggle/input/extracting-attributes-from-fashion-images-2/test',
    target_size=(100,100),
    batch_size=32,
    shuffle=False
)
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)
sub = pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-2/sample_submission.csv')
sub['label'] = pred
sub.to_csv('/kaggle/working/file1.csv')


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Found 14712 validated image filenames belonging to 7 classes.
Found 3679 validated image filenames belonging to 7 classes.
Epoch 1/20
460/460 [==============================] - 121s 243ms/step - loss: 3.7002 - accuracy: 0.2984 - val_loss: 1.8180 - val_accuracy: 0.2955
Epoch 2/20
460/460 [==============================] - 71s 153ms/step - loss: 1.6205 - accuracy: 0.3580 - val_loss: 1.8159 - val_accuracy: 0.2955
Epoch 3/20
460/460 [==============================] - 69s 150ms/step - loss: 1.4733 - accuracy: 0.4190 - val_loss: 1.8169 - val_accuracy: 0.3264
Epoch 4/20
460/460 [==============================] - 61s 133ms/step - loss: 1.3319 - accuracy: 0.4784 - val_loss: 1.8275 - val_accuracy: 0.2191
Epoch 5/20
460/460 [==============================] - 63s 136ms/step - loss: 1.2003 - accuracy: 0.5349 - val_loss: 1.8382 - val_accuracy: 0.2191
Epoch 6/20
460/460 [==============================] - 62s 134ms/step - loss: 1.1062 - accuracy: 0.5695 - val_loss: 1.8438 - val_accuracy: 0.2191
Epoch 